In [10]:
# %pip install pygame

In [1]:
import numpy as np

In [2]:
class PacManZ:
    def __init__(self, board_size=(15, 10), num_zombies=4, num_obstacles=10, 
                 num_vaccines=1, num_pits=1):
        self.board_size = board_size
        self.num_zombies = num_zombies
        self.num_obstacles = num_obstacles
        self.num_vaccines = num_vaccines
        self.num_pits = num_pits
        self.reset()
        
    def reset(self):
        self.player_pos = (np.random.randint(0, self.board_size[0]), 
                           np.random.randint(0, self.board_size[1]))
        self.zombie_positions = [(np.random.randint(0, self.board_size[0]), 
                            np.random.randint(0, self.board_size[1])) 
                           for i in range(self.num_zombies)]
        self.obstacle_pos = [(np.random.randint(0, self.board_size[0]), 
                              np.random.randint(0, self.board_size[1])) 
                             for i in range(self.num_obstacles)]
        self.vaccine_pos = [(np.random.randint(0, self.board_size[0]), 
                             np.random.randint(0, self.board_size[1])) 
                            for i in range(self.num_vaccines)]
        self.pit_pos = [(np.random.randint(0, self.board_size[0]), 
                         np.random.randint(0, self.board_size[1])) 
                        for i in range(self.num_pits)]
        self.vaccine_count = self.num_vaccines
        self.shots_remaining = 3
        self.game_over = False
        self.zombie_cured = [False] * self.num_zombies
        self._update_zombie_positions()

    def _update_zombie_positions(self):
        new_positions = []
        for i, pos in enumerate(self.zombie_positions):
            move = np.random.choice(["up", "down", "left", "right"])
            new_pos = pos.copy()
            if move == "up":
                new_pos[0] -= 1
            elif move == "down":
                new_pos[0] += 1
            elif move == "left":
                new_pos[1] -= 1
            elif move == "right":
                new_pos[1] += 1
            new_positions.append(new_pos)
        self.zombie_positions = new_positions
    
    def _update_zombie_positions(self):
        for i, zombie_pos in enumerate(self.zombie_positions):
            move = np.random.choice(["up", "down", "left", "right"])
            # Get the row and column of the zombie's current position
            row, col = zombie_pos

            # Determine the new position of the zombie based on its current direction
            if move == 'right':
                new_pos = (row, col + 1)
            elif move == 'left':
                new_pos = (row, col - 1)
            elif move == 'up':
                new_pos = (row - 1, col)
            elif move == 'down':
                new_pos = (row + 1, col)

            # Check if the new position is within the board boundaries
            if new_pos[0] < 0 or new_pos[0] >= self.board_size[0] or new_pos[1] < 0 or new_pos[1] >= self.board_size[1]:
                continue

            # Check if the new position is a pit
            if new_pos in self.pit_pos:
                continue

            # Update the zombie's position
            self.zombie_positions[i] = new_pos
        
    def get_successor_state(self, position, action):
        if action == 'up':
            next_position = (position[0]-1, position[1])
        elif action == 'down':
            next_position = (position[0]+1, position[1])
        elif action == 'left':
            next_position = (position[0], position[1]-1)
        elif action == 'right':
            next_position = (position[0], position[1]+1)
        else:
            raise ValueError('Invalid action.')
        if next_position[0] < 0 or next_position[0] >= self.board_size[0] or \
           next_position[1] < 0 or next_position[1] >= self.board_size[1]:
            return position
        if next_position in self.obstacle_pos:
            return position
        return next_position
    
    def get_valid_actions(self, position):
        valid_actions = []
        if position[0] > 0 and (position[0]-1, position[1]) not in self.obstacle_pos:
            valid_actions.append('up')
        if position[0] < self.board_size[0]-1 and (position[0]+1, position[1]) not in self.obstacle_pos:
            valid_actions.append('down')
        if position[1] > 0 and (position[0], position[1]-1) not in self.obstacle_pos:
            valid_actions.append('left')
        if position[1] < self.board_size[1]-1 and (position[0], position[1]+1) not in self.obstacle_pos:
            valid_actions.append('right')
        return valid_actions
    
    def perform_action(self, action):
        if self.game_over:
            raise ValueError('Game is over.')
        reward = 0
        done = False
        # Update player position
        if action in self.get_valid_actions(self.player_pos):
            self.board[self.player_pos] = 0  # Remove player from current position
            self.player_pos = self.get_successor_state(self.player_pos, action)
            self.board[self.player_pos] = 1  # Place player in new position
            
            # Check if player falls into the pit
            if self.board[self.player_pos] == self.PIT:
                reward = -1000
                done = True
                return self.get_state(), reward, done
            
            # Check if player is captured by zombies
            if self.is_over(self.player_pos, self.zombie_positions):
                reward = -1000
                done = True
                return self.get_state(), reward, done
            
            # Check if player collects vaccine
            if self.board[self.player_pos] == self.VACCINE:
                self.vaccine_count += 1
                self.board[self.player_pos] = 0  # Remove vaccine from board
                
                # Check if player wins the game
                if self.vaccine_count == self.MAX_VACCINE_COUNT:
                    reward = 1000
                    done = True
                    return self.get_state(), reward, done
            
            # Move zombies
            for i in range(self.num_zombies):
                # Choose a random action for the zombie
                zombie_action = np.random.choice(self.get_valid_actions(self.zombie_positions[i]))
                self.board[self.zombie_positions[i]] = 0  # Remove zombie from current position
                self.zombie_positions[i] = self.get_successor_state(self.zombie_positions[i], zombie_action)
                self.board[self.zombie_positions[i]] = -1  # Place zombie in new position
            
            # Check if player is captured by zombies after zombies move
            if self.is_over(self.player_pos, self.zombie_positions):
                reward = -1000
                done = True
                return self.get_state(), reward, done
            
            return self.get_state(), reward, done
            
        else:
            # Invalid action
            return self.get_state(), reward, done
        
    def get_state(self):
        # Get positions of player, zombies, and pits
        player_pos = self.player_pos
        zombie_pos = self.zombie_positions
        pit_pos = self.pits
        
        # Convert positions to binary arrays
        player_pos_arr = np.zeros(self.board_size)
        player_pos_arr[player_pos[0], player_pos[1]] = 1
        zombie_pos_arr = np.zeros(self.board_size)
        for zombie in zombie_pos:
            zombie_pos_arr[zombie[0], zombie[1]] = 1
        pit_pos_arr = np.zeros(self.board_size)
        for pit in pit_pos:
            pit_pos_arr[pit[0], pit[1]] = 1
        
        # Concatenate binary arrays into a feature vector
        state = np.concatenate((player_pos_arr.flatten(),
                                zombie_pos_arr.flatten(),
                                pit_pos_arr.flatten()))
        
        return state
    
    def render(self):
        # Create a copy of the board with all 0s
        board_copy = np.zeros(self.board_size)
        
        # Set the player's position to 1
        board_copy[self.player_pos[0], self.player_pos[1]] = 1
        
        # Set the zombies' positions to 2
        for zombie_pos in self.zombie_positions:
            board_copy[zombie_pos[0], zombie_pos[1]] = 2
        
        # Set the pits' positions to 3
        for pit_pos in self.pits:
            board_copy[pit_pos[0], pit_pos[1]] = 3
        
        # Set the bullets' positions to 4
        for bullet_pos in self.bullet_positions:
            board_copy[bullet_pos[0], bullet_pos[1]] = 4
        
        # Set the exit's position to 5
        board_copy[self.exit_pos[0], self.exit_pos[1]] = 5
        
        # Convert the board to a string representation for display
        board_str = ""
        for i in range(self.board_size[0]):
            for j in range(self.board_size[1]):
                if board_copy[i,j] == 0:
                    board_str += ". "
                elif board_copy[i,j] == 1:
                    board_str += "P "
                elif board_copy[i,j] == 2:
                    board_str += "Z "
                elif board_copy[i,j] == 3:
                    board_str += "X "
                elif board_copy[i,j] == 4:
                    board_str += "* "
                elif board_copy[i,j] == 5:
                    board_str += "E "
            board_str += "\n"
            
        # Print the board
        print(board_str)

In [13]:
import numpy as np
import random
import pygame

# Define the linear function approximator
def value_approximator(state, weights):
    return np.dot(state, weights)

# Define the feature vector for the state
def feature_extractor(state):
    # TODO: Define the feature vector for the current state
    pass

# Define the Q-learning algorithm
def q_learning(num_episodes, alpha, gamma):
    # Initialize weights and the game environment
    weights = np.zeros(num_features)
    pygame.init()
    game = PacManZ()
    
    for episode in range(num_episodes):
        # Reset the game environment
        game.reset()

        # Loop until the game is over
        while not game.is_over():
            # Get the current state
            state = feature_extractor(game.get_state())

            # Choose the best action based on the current weights
            actions = game.get_valid_actions()
            q_values = [value_approximator(feature_extractor(game.get_successor_state(action)), weights) for action in \
                        actions]
            best_action = actions[np.argmax(q_values)]

            # Take the chosen action and get the next state and reward
            next_state, reward = game.perform_action(best_action)

            # Update the weights using the Q-learning update rule
            if game.is_over():
                target = reward
            else:
                next_q_values = [value_approximator(feature_extractor(game.get_successor_state(action)), weights) for action in actions]
                target = reward + gamma * np.max(next_q_values)
            error = target - value_approximator(state, weights)
            weights += alpha * error * state

    return weights

# Use the learned weights to play the game
def play(weights):
    # Initialize the game environment
    pygame.init()
    game = PacManZ()

    # Loop until the game is over
    while not game.is_over():
        # Get the current state
        state = feature_extractor(game.get_state())

        # Choose the best action based on the learned weights
        actions = game.get_valid_actions()
        q_values = [value_approximator(feature_extractor(game.get_successor_state(action)), weights) for action in actions]
        best_action = actions[np.argmax(q_values)]

        # Take the chosen action and render the game
        game.perform_action(best_action)
        game.render()

    # Quit pygame
    pygame.quit()

# Example usage
num_features = 100  # TODO: Set the number of features
alpha = 0.1         # TODO: Set the learning rate
gamma = 0.9         # TODO: Set the discount factor
num_episodes = 1000 # TODO: Set the number of training episodes

# Train the algorithm
weights = q_learning(num_episodes, alpha, gamma)

# Use the learned weights to play the game
play(weights)

AttributeError: 'PacManZ' object has no attribute 'is_over'

In [2]:
import pygame

# Define some colors
BLACK = (0, 0, 0)
GRAY = (60, 60, 60)
RED = (255, 0, 0)

# Set the dimensions of the board
BOARD_WIDTH = 600
BOARD_HEIGHT = 400

# Set the dimensions of each cell
CELL_WIDTH = BOARD_WIDTH // 15
CELL_HEIGHT = BOARD_HEIGHT // 10

# Load the images
def load_img(path, padding=10):        
    img = pygame.image.load(path)
    # Scale the image to fit inside a cell
    scaled_image_width = CELL_WIDTH - padding # subtract some padding
    scaled_image_height = CELL_HEIGHT - padding # subtract some padding
    img = pygame.transform.scale(img, (scaled_image_width, scaled_image_height))
    return img

zombie = load_img("img/zombie.png")
player = load_img("img/player.png")
pit = load_img("img/pit2.png")
endpoint = load_img("img/endpoint.png")
vaccine = load_img("img/vaccine.png", padding=15)
obstacle = load_img("img/obstacle3.png", padding=0)

# Initialize Pygame
pygame.init()

# Set up the display
screen = pygame.display.set_mode((BOARD_WIDTH, BOARD_HEIGHT))
pygame.display.set_caption("Pac-man Z edition")

# Create the board
board = []
for row in range(10):
    board.append([])
    for column in range(15):
        board[row].append(0)

# Set up the font for the numbers on the board
font = pygame.font.SysFont('Calibri', 25, True, False)

# Draw the board
done = False
while not done:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            done = True

    # Draw the cells
    for row in range(10):
        for column in range(15):
            color = GRAY
            pygame.draw.rect(screen, color, [column * CELL_WIDTH, row * CELL_HEIGHT, CELL_WIDTH, CELL_HEIGHT], 1) # border of 1px

    # Draw the lines between cells
    for x in range(0, BOARD_WIDTH, CELL_WIDTH):
        pygame.draw.line(screen, BLACK, (x, 0), (x, BOARD_HEIGHT), 1) # vertical lines
    for y in range(0, BOARD_HEIGHT, CELL_HEIGHT):
        pygame.draw.line(screen, BLACK, (0, y), (BOARD_WIDTH, y), 1) # horizontal lines

    # Draw the image in cell (5, 7)
    center_x = 0 * CELL_WIDTH + CELL_WIDTH // 2
    center_y = 1 * CELL_HEIGHT + CELL_HEIGHT // 2
    image_rect = zombie.get_rect()
    image_rect.center = (center_x, center_y)
    screen.blit(zombie, image_rect)

    # Draw the image in cell (5, 7)
    center_x = 5 * CELL_WIDTH + CELL_WIDTH // 2
    center_y = 4 * CELL_HEIGHT + CELL_HEIGHT // 2
    image_rect = player.get_rect()
    image_rect.center = (center_x, center_y)
    screen.blit(player, image_rect)

    # Draw the image in cell (5, 7)
    center_x = 7 * CELL_WIDTH + CELL_WIDTH // 2
    center_y = 2 * CELL_HEIGHT + CELL_HEIGHT // 2
    image_rect = vaccine.get_rect()
    image_rect.center = (center_x, center_y)
    screen.blit(vaccine, image_rect)

    # Draw the image in cell (5, 7)
    center_x = 5 * CELL_WIDTH + CELL_WIDTH // 2
    center_y = 1 * CELL_HEIGHT + CELL_HEIGHT // 2
    image_rect = pit.get_rect()
    image_rect.center = (center_x, center_y)
    screen.blit(pit, image_rect)

    # Draw the image in cell (5, 7)
    center_x = 7 * CELL_WIDTH + CELL_WIDTH // 2
    center_y = 6 * CELL_HEIGHT + CELL_HEIGHT // 2
    image_rect = obstacle.get_rect()
    image_rect.center = (center_x, center_y)
    screen.blit(obstacle, image_rect)

    # Draw the image in cell (5, 7)
    center_x = 3 * CELL_WIDTH + CELL_WIDTH // 2
    center_y = 1 * CELL_HEIGHT + CELL_HEIGHT // 2
    image_rect = obstacle.get_rect()
    image_rect.center = (center_x, center_y)
    screen.blit(obstacle, image_rect)

    # Draw the image in cell (5, 7)
    center_x = 14 * CELL_WIDTH + CELL_WIDTH // 2
    center_y = 6 * CELL_HEIGHT + CELL_HEIGHT // 2
    image_rect = endpoint.get_rect()
    image_rect.center = (center_x, center_y)
    screen.blit(endpoint, image_rect)

    # Draw the numbers on the board
    for row in range(10):
        for column in range(15):
            if board[row][column] != 0:
                text = font.render(str(board[row][column]), True, BLACK)
                text_rect = text.get_rect()
                text_rect.center = (column * CELL_WIDTH + CELL_WIDTH // 2, row * CELL_HEIGHT + CELL_HEIGHT // 2)
                screen.blit(text, text_rect)

    # Update the screen
    pygame.display.flip()

# Quit Pygame
pygame.quit()